# Naive Bayes

Import the Final Data

In [402]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [403]:
import pandas as pd
path = "/content/drive/MyDrive/Text Mining/project_newsapi_final.csv"
df = pd.read_csv(path)
df.head()

,Title,Source,Author,Summary,Date,Text,Domain,Country,News Source,AllSides Bias Rating,Agree/Disagree,POS_tag,Title_nltk,Title_vader,Summary_nltk,Summary_vader,Text_nltk,Text_vader
0,OpenAI boss ‘heartened’ by talks with world le...,Associated Press,Tia Goldenberg,"TEL AVIV, Israel (AP) — OpenAI CEO Sam Altman ...",2023-06-05,OpenAI boss ‘heartened’ by talks with world le...,apnews.com,US,Associated Press,Lean Left,34008/26698,"[('OpenAI', 'NNP'), ('boss', 'NN'), ('‘', 'NN'...",0.000000,-0.2732,-0.600000,0.7351,0.039016,0.9945
1,Is it real or made by AI? Europe wants label a...,Associated Press,Kelvin Chan,LONDON (AP) — The European Union is pushing on...,2023-06-06,Is it real or made by AI? Europe wants a label...,apnews.com,US,Associated Press,Lean Left,34008/26698,"[('Is', 'VBZ'), ('it', 'PRP'), ('real', 'JJ'),...",0.200000,-0.4019,-0.125000,0.5859,-0.029093,0.9932
2,Mass event will let hackers test limits of new...,Associated Press,Matt O'brien,Mass event will let hackers test limits of new...,2023-05-11,"Rumman Chowdhury, co-founder of Humane Intelli...",apnews.com,US,Associated Press,Lean Left,34008/26698,"[('Rumman', 'NNP'), ('Chowdhury', 'NNP'), (','...",0.136364,0.0000,0.054545,-0.7034,0.073494,0.9786
3,How Europe is leading world in building guardr...,Associated Press,ap news,LONDON (AP) — Authorities around the world are...,2023-05-11,FILE - Text from the ChatGPT page of the OpenA...,apnews.com,US,Associated Press,Lean Left,34008/26698,"[('FILE', 'NNP'), ('-', ':'), ('Text', 'NN'), ...",0.000000,0.0000,-0.033333,0.4767,0.009417,0.9862
4,"The cyber gulag: How Russia tracks, censors, c...",Associated Press,Dasha Litvinova,"TALLINN, Estonia (AP) — When Yekaterina Maksim...",2023-05-23,Yekaterina Maksimova enters a Moscow subway st...,apnews.com,US,Associated Press,Lean Left,34008/26698,"[('Yekaterina', 'NNP'), ('Maksimova', 'NNP'), ...",0.000000,-0.2960,0.037500,0.3804,0.041747,-0.9903


In [391]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
)
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

Select Features

In [404]:
# Summary, Text, Title, Title NLTK, Title Vader, Vader Title, Vader Text, Vader Summary
for col in df.columns:
    print(col)

Title
Source
Author
Summary
Date
Text
Domain
Country
News Source
AllSides Bias Rating
Agree/Disagree
POS_tag
Title_nltk
Title_vader
Summary_nltk
Summary_vader
Text_nltk
Text_vader


In [405]:
df = df.loc[:,['Title', 'Summary', 'Text', 'Title_nltk','Title_vader','Summary_nltk','Summary_vader','Text_nltk','Text_vader',]]
df.dtypes


Title             object
Summary           object
Text              object
Title_nltk       float64
Title_vader      float64
Summary_nltk     float64
Summary_vader    float64
Text_nltk        float64
Text_vader       float64
dtype: object

In [406]:
from pandas.api.types import is_numeric_dtype

def PrintRange(colStr):
  if(is_numeric_dtype(df[colStr])):
    print(colStr)
    totRange = df[colStr].max() - df[colStr].min()
    print(totRange)

def GetRange(colStr):
  if(is_numeric_dtype(df[colStr])):
    totRange = df[colStr].max() - df[colStr].min()
    min = df[colStr].min()
    max = df[colStr].max()
    midPoint = min + (totRange/2)
    return [totRange, min, max, midPoint]

def GetNumericColumns(df):
  colNames = df.columns
  numColNames = []
  for col in colNames:
    if(is_numeric_dtype(df[col])):
      numColNames.append(col)
  return numColNames

def GetStartingPoint(d_f):
  rangeDataDict = {}
  for col in d_f.columns:
    if(is_numeric_dtype(d_f[col])):
      print(col)
      vals = GetRange(col)
      print("Total Range, Min, Max, MidPoint",GetRange(col))
      rangeDataDict[col] = vals

  return rangeDataDict

def CreateLabes(df):
# Create Starting Labels
  startingLabelDict = GetStartingPoint(df)

  cols = GetNumericColumns(df)
  print(cols)
  labelDict = {}
  for col in cols:
    key = col + '_label'
    labelDict[key] = []

  # iterate through DF Row By Row, Construct Label lists containing labels
  for index, row in df.iterrows():
    #Iterate through Label Names add to the list
    for numCol in cols:
      thresh = startingLabelDict[numCol][2]
      if row[numCol] >= thresh:
        # Positive Sentiment
        labelDict[numCol + '_label'].insert(index, 'Positive')
      if row[numCol] < thresh:
        # Negative Sentiment
        labelDict[numCol + '_label'].insert(index, 'Negative')
  # Concatenate with Dataframe
  for key in labelDict.keys():
    df[key]= labelDict[key]
    df[key] = df[key].astype("category")

  return df


In [407]:
def CreateXLabs(df, labelColumnStr, textColStr):
  vectorizer = TfidfVectorizer()
  textArr = vectorizer.fit_transform(df[textColStr].tolist())
  bow = np.array(textArr.todense())

  colnames = vectorizer.get_feature_names_out()

  revDf = pd.DataFrame(textArr.toarray(), columns=colnames)

  x = csr_matrix(revDf.values)
  labels = df[labelColumnStr]
  return x, labels

In [408]:
dfLabs = CreateLabes(df)
print(dfLabs.dtypes)
df.head()
x, labels = CreateXLabs(dfLabs, "Title_vader", "Title")

Title_nltk
Total Range, Min, Max, MidPoint [2.0, -1.0, 1.0, 0.0]
Title_vader
Total Range, Min, Max, MidPoint [1.7958, -0.8979, 0.8979, 0.0]
Summary_nltk
Total Range, Min, Max, MidPoint [1.7000000000000002, -0.8, 0.9, 0.050000000000000044]
Summary_vader
Total Range, Min, Max, MidPoint [1.8235000000000001, -0.9169, 0.9066, -0.005149999999999988]
Text_nltk
Total Range, Min, Max, MidPoint [0.95, -0.4499999999999999, 0.5, 0.025000000000000078]
Text_vader
Total Range, Min, Max, MidPoint [2.0, -1.0, 1.0, 0.0]
['Title_nltk', 'Title_vader', 'Summary_nltk', 'Summary_vader', 'Text_nltk', 'Text_vader']
Title                    object
Summary                  object
Text                     object
Title_nltk              float64
Title_vader             float64
Summary_nltk            float64
Summary_vader           float64
Text_nltk               float64
Text_vader              float64
Title_nltk_label       category
Title_vader_label      category
Summary_nltk_label     category
Summary_vader_labe

In [400]:
X_train, X_test, y_train, y_test = train_test_split(x, labels,
                                                    test_size=0.2)

Fit the Model

In [401]:
modelSent = MultinomialNB().fit(X_train, y_train)
y_pred = modelSent.predict(X_test)

ValueError: ignored

Create the Confusion Matrix

In [ ]:
cmlables = ["Positive", "Negative"]
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cmlables)
disp.plot()

Check Accuracy

In [ ]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))